# Drowsiness Detection

Drowsiness detection is straight forward. 

1. Find face in the frame
2. Apply landmark detection on it
3. Extract the eyes
4. Compute the ratio between eyelids to measure either open or close
5. Sound an alarm if the eyes have been closed for a sufficentely long time


## Let's get to the code

In [1]:
# import necessary packages
from scipy.spatial import distance as dist
from imutils import face_utils
from imutils.video import VideoStream
from threading import Thread
import numpy as np
import imutils
import time
import dlib
import cv2

`sound_alarm` is a helper function to play an alarm sound

In [32]:
import playsound

# play an alarm sound 
def sound_alarm(path):
    playsound.playsound(path)  # block=False -> play asynchronously 


`eye_aspect_ratio` is a helper method to calculate the aspect ratio of eyes

To visualize conside the following image from Soukupová and Čech’s 2016 paper Real-Time Eye Blink Detection using Facial Landmarks:

<img src="https://www.pyimagesearch.com/wp-content/uploads/2017/04/blink_detection_plot.jpg">

In [3]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of 
    # vertical eye landmarks (x,y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    
    # compute the euclidean distance between the horizonatal
    # eye landmark
    C = dist.euclidean(eye[0], eye[3])
    
    # compute the eye aspect ratio(ear) 
    ear = (A+B)/(2*C)
    
    # return the eye aspect ratio
    return ear

Now, we'll define a few important constatnts that may need to be ___tuned to achieve better results___.

In [4]:
# define two constants, one for the EAR to indicate 
# blink and then a second constant for the number of 
# consecutive frames the eye must be below the threshold 
# for the alarm
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 48

# initialize the frame counter as well as the a bool used to
# indicate if the alarm is going off
COUNTER = 0
ALARM_ON = False

In [5]:
# paths to model
path_to_landmark_model = "..\\DNN_MODELS\\shape_predictor_68_face_landmarks.dat"

For simplicity and speed we'll use dlib's HOG-based face detector, ___but the DNN model is much more robust and should be used instead___.

In [21]:
# initialize dlib's face detector(HOG based) and the facial
# landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(path_to_landmark_model)

[INFO] loading facial landmark predictor...


To visualize the landmarks to extract look at the following image:

<img src="https://www.pyimagesearch.com/wp-content/uploads/2017/04/facial_landmarks_68markup-768x619.jpg">


Therefore, to extract the left and right eyes we simply need to slice the array

In [22]:
# grab the indices of the left and right eye
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]

Now, we can finally get to the main loop of the program

In [23]:
# import this to display images like a video in notebook
from IPython.display import display , Image, clear_output 

In [37]:
# start the video stream thread
print("[INFO] starting video stream thread...")
vs = VideoStream(src=0).start()
time.sleep(1.0) # let frames buffer

# loop over the frames
while True:
    # grab the frame from the threaded video file stream, resize
    # it an convert it to gray scale
    frame = vs.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect faces in grayscale image
    rects = detector(gray, upsample_num_times=0)
    
    # loop over the face detections 
    for rect in rects:
        # determine the facial landmarks for the face region, then 
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        # extract the left and right eye coordinates, then use the
        # coordinates to compute the eye aspect ratio for both eyes
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        # Soukupová and Čech recommend averaging both eye aspect 
        # ratios together to obtain a better estimation
        ear = (leftEAR + rightEAR) / 2.0
        
        # compute the convex hull for both left and right eye, then
        # visualize each of the eyes with a green outline
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0,255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull],-1, (0,255, 0), 1)
        
        # now check if the person in the frame is dozing off
        if ear < EYE_AR_THRESH: 
            # eyes closed
            COUNTER+=1
            
            
            # if the eyes were closed for a sufficent number of frames
            # then sound the alarm
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                # if the alarm is not on, turn it on
                if not ALARM_ON:
                    ALARM_ON = True
                   
                    # play alarm, on background thread
                    t = Thread(target=sound_alarm, args=("alarm.wav",))
                    t.deamon = True
                    t.start()
                    
                    # draw an alarm on the frame
                    cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255),2)
            
            # otherwise, the EAR is not below the blink thresh i.e. eyes
            # open , so reset the counter and alarm
        else:
            COUNTER = 0
            ALARM_ON = False
                
        # draw an counter on the frame
        cv2.putText(frame, "{}".format(COUNTER), (10, 300),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255),2)
            
                
        # draw the computed eye aspect ratio on the frame to help
        # with debugging and setting the correct eye aspect ratio
        # thresholds and frame counters
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30), 
                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        
           
            
    # for jupyter first we need to save the image then dsplay it 
    # for a video like effect
    cv2.imwrite('pic.jpg', frame) # first we save it
    display(Image("pic.jpg"))

    clear_output(wait=True)  
        

KeyboardInterrupt: 

In [35]:
# do a bit of cleanup
COUNTER=0
vs.stop()